<a href="https://colab.research.google.com/github/NullV4/bd/blob/main/RDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from pyspark import SparkContext

In [3]:
sc = SparkContext("local", "FinanceAnalysis")

In [4]:
ventas_data = [
    ("Producto_A", 10000, "2024-01"), ("Producto_B", 15000, "2024-01"), ("Servicio_X", 7000, "2024-01"),
    ("Producto_A", 12000, "2024-02"), ("Producto_B", 16000, "2024-02"), ("Servicio_X", 8000, "2024-02"),
    ("Producto_A", 14000, "2024-03"), ("Producto_B", 17000, "2024-03"), ("Servicio_X", 9000, "2024-03"),
    ("Producto_C", 20000, "2024-01"), ("Producto_C", 21000, "2024-02"), ("Producto_C", 22000, "2024-03"),
]

ventas_rdd = sc.parallelize(ventas_data)


In [5]:
costos_data = [
    ("Producto_A", 4000), ("Producto_B", 6000), ("Servicio_X", 3000), ("Producto_C", 10000),
    ("Producto_A", 4500), ("Producto_B", 6500), ("Servicio_X", 3500), ("Producto_C", 10500),
    ("Producto_A", 5000), ("Producto_B", 7000), ("Servicio_X", 4000), ("Producto_C", 11000),
]
costos_rdd = sc.parallelize(costos_data)

In [6]:

gastos_data = [
    ("Infraestructura", 2000), ("Tecnología", 3000), ("Marketing", 1500), ("Otros", 1000),
    ("Infraestructura", 2100), ("Tecnología", 3200), ("Marketing", 1600), ("Otros", 1100),
    ("Infraestructura", 2200), ("Tecnología", 3300), ("Marketing", 1700), ("Otros", 1200),
]
gastos_rdd = sc.parallelize(gastos_data)

In [7]:
flujo_data = [
    ("Ingresos", 25000), ("Egresos", 18000), ("Ingresos", 27000), ("Egresos", 19000),
    ("Ingresos", 28000), ("Egresos", 20000), ("Ingresos", 30000), ("Egresos", 21000),
]
flujo_rdd = sc.parallelize(flujo_data)

In [8]:
inversion_data = [
    ("Proyecto_NuevaTecnologia", 10000), ("Proyecto_DesarrolloProducto", 15000),
    ("Proyecto_MejoraPlataforma", 8000), ("Proyecto_InvestigacionIA", 12000),
    ("Proyecto_MejoraUIUX", 6000), ("Proyecto_ExpansionInfraestructura", 14000),
]

inversion_rdd = sc.parallelize(inversion_data)

In [9]:
ventas_totales = ventas_rdd.map(lambda x: (x[0], x[1])).reduceByKey(lambda x, y: x + y)
print("Ventas Totales por Producto/Servicio:", ventas_totales.collect())

Ventas Totales por Producto/Servicio: [('Producto_A', 36000), ('Producto_B', 48000), ('Servicio_X', 24000), ('Producto_C', 63000)]


In [10]:
margen_rdd = ventas_rdd.map(lambda x: (x[0], x[1])).join(costos_rdd).mapValues(lambda x: x[0] - x[1])
margen_total_por_producto = margen_rdd.reduceByKey(lambda x, y: x + y)
print("Margen Total de Ganancia por Producto/Servicio:", margen_total_por_producto.collect())

Margen Total de Ganancia por Producto/Servicio: [('Producto_B', 85500), ('Producto_A', 67500), ('Servicio_X', 40500), ('Producto_C', 94500)]


In [11]:
flujo_total = flujo_rdd.reduceByKey(lambda x, y: x + y)
print("Flujo de Caja:", flujo_total.collect())

Flujo de Caja: [('Ingresos', 110000), ('Egresos', 78000)]


In [12]:
inversion_total = inversion_rdd.map(lambda x: x[1]).sum()
print("Inversión Total en Innovación:", inversion_total)

Inversión Total en Innovación: 65000


In [13]:
gastos_totales = gastos_rdd.map(lambda x: x[1]).sum()
print("Gastos Generales y Administrativos Totales:", gastos_totales)

Gastos Generales y Administrativos Totales: 23900


In [14]:
ventas_total_mensual = ventas_rdd.map(lambda x: x[1]).sum()
ratio_inversion = inversion_total / ventas_total_mensual
print("Ratio de Inversión en Innovación:", ratio_inversion)

Ratio de Inversión en Innovación: 0.38011695906432746


In [15]:
margen_promedio_por_producto = margen_total_por_producto.mapValues(lambda x: x / 3)  # Asumiendo 3 meses
print("Margen Bruto Promedio por Producto:", margen_promedio_por_producto.collect())

Margen Bruto Promedio por Producto: [('Producto_B', 28500.0), ('Producto_A', 22500.0), ('Servicio_X', 13500.0), ('Producto_C', 31500.0)]


In [16]:
flujo_mensual = flujo_rdd.map(lambda x: ("Ingresos" if x[0] == "Ingresos" else "Egresos", x[1])).reduceByKey(lambda x, y: x + y)
print("Total de Ingresos y Egresos:", flujo_mensual.collect())

Total de Ingresos y Egresos: [('Ingresos', 110000), ('Egresos', 78000)]


In [18]:
costos_promedio = costos_rdd.mapValues(lambda x: x / 3).reduceByKey(lambda x, y: x + y)  # Asumiendo 3 meses
print("Costos Promedio por Producto:", costos_promedio.collect())

Costos Promedio por Producto: [('Producto_A', 4500.0), ('Producto_B', 6500.0), ('Servicio_X', 3500.0), ('Producto_C', 10500.0)]


In [19]:
ingresos_netos = flujo_total.filter(lambda x: x[0] == "Ingresos").map(lambda x: x[1]).sum() - flujo_total.filter(lambda x: x[0] == "Egresos").map(lambda x: x[1]).sum()
print("Ingresos Netos Totales:", ingresos_netos)

Ingresos Netos Totales: 32000


In [20]:
producto_max_margen = margen_total_por_producto.takeOrdered(1, key=lambda x: -x[1])
print("Producto/Servicio con Mayor Margen de Ganancia:", producto_max_margen)

Producto/Servicio con Mayor Margen de Ganancia: [('Producto_C', 94500)]


In [21]:
inversion_ordenada = inversion_rdd.sortBy(lambda x: x[1], ascending=False)
print("Inversiones Ordenadas por Proyecto:", inversion_ordenada.collect())

Inversiones Ordenadas por Proyecto: [('Proyecto_DesarrolloProducto', 15000), ('Proyecto_ExpansionInfraestructura', 14000), ('Proyecto_InvestigacionIA', 12000), ('Proyecto_NuevaTecnologia', 10000), ('Proyecto_MejoraPlataforma', 8000), ('Proyecto_MejoraUIUX', 6000)]


In [22]:
total_gastos = gastos_rdd.map(lambda x: x[1]).sum()
porcentaje_gastos = gastos_rdd.map(lambda x: (x[0], (x[1] / total_gastos) * 100))
print("Porcentaje de Gastos por Categoría:", porcentaje_gastos.collect())

Porcentaje de Gastos por Categoría: [('Infraestructura', 8.368200836820083), ('Tecnología', 12.552301255230125), ('Marketing', 6.2761506276150625), ('Otros', 4.184100418410042), ('Infraestructura', 8.786610878661087), ('Tecnología', 13.389121338912133), ('Marketing', 6.694560669456067), ('Otros', 4.602510460251046), ('Infraestructura', 9.205020920502092), ('Tecnología', 13.807531380753138), ('Marketing', 7.112970711297072), ('Otros', 5.02092050209205)]
